In [72]:
from BorderQuery import query_mungedata, insert_predictions, query_features
from BorderModel import BorderData, handle_categoricals
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.grid_search import GridSearchCV

In [7]:
df = query_mungedata(2, 1, '2008-1-1')

## Sanity test model

In [19]:
from BorderModel import BorderData
data = BorderData(df, categoricals=['event'])

model = ExtraTreesRegressor(n_jobs=-1, n_estimators=16, bootstrap=True, oob_score=True)
params = {}
grid = GridSearchCV(model, params, cv=data.cv_train)
grid.fit(data.X_train, data.y_train)

data.predict(grid)
data.predict_ensemble()
data.print_metrics(grid)

OOB:  0.87593391716
Best score:  0.344903637231
** MSE for last cv fold **
Baseline :  96.7590735774
Model    :  111.068893622
Ensemble :  91.7590631555
Weights  :  (0.77324623529285852, 1.2137065911541605)
** R^2 for last cv fold **
Baseline :  0.14722281598
Model    :  0.0211045348705
Ensemble :  0.191289947362
** Explained variance for last cv fold **
Baseline :  0.304333505904
Model    :  0.239870438019
Ensemble :  0.339796605641


/Users/jng/anaconda/lib/python2.7/site-packages/sklearn/ensemble/forest.py:687: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


## Predict on full training data

In [54]:
model = ExtraTreesRegressor(n_jobs=-1, n_estimators=16, bootstrap=True, oob_score=True)
params = {}
grid = GridSearchCV(model, params, cv=data.cv_train)
grid.fit(data.X, data.y)

GridSearchCV(cv=array([[ [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 7...20, 108021, 108022, 108023, 108024, 108025, 108026, 108027, 108028, 108029, 108030]]], dtype=object),
       error_score='raise',
       estimator=ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=2, min_weight_fraction_leaf=0.0,
          n_estimators=16, n_jobs=-1, oob_score=True, random_state=None,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

# TODO: handle event categoricals

In [55]:
features = query_features('2015-12-1', '2018-1-1')

In [56]:
fmatrix = handle_categoricals(features, ['event'])

In [57]:
yhat = grid.best_estimator_.predict(fmatrix)

In [68]:
fmatrix.index.date

array([datetime.date(2015, 12, 1), datetime.date(2015, 12, 1),
       datetime.date(2015, 12, 1), ..., datetime.date(2017, 12, 31),
       datetime.date(2017, 12, 31), datetime.date(2017, 12, 31)], dtype=object)

In [73]:
insert_predictions('v1.0', 2, 1, fmatrix.index.date, yhat)

IntegrityError: duplicate key value violates unique constraint "predictions_pkey"
DETAIL:  Key (model_version, munger_id, crossing_id, date)=(v1.0, 2, 1, 2015-12-01 00:00:00) already exists.
